In [2]:
#==================================================================
#Program: SLADS_multichannel
#Author(s): David Helminiak
#Date Created: 4 October 2019
#Date Last Modified: November 2019
#Changelog: 0.1  - Migration, Integration for multichannel values - Oct. 2019
#
#==================================================================

#==================================================================
#ADDITIONAL NOTES:
#==================================================================
#Add Breakpoint anywhere in the program: 
#Tracer()()
#
#==================================================================

#==================================================================
#LIBRARY IMPORTS
#==================================================================
from __future__ import absolute_import, division, print_function
#import tensorflow as tf
#tf.enable_eager_execution() #Evaluate all operations without building graphs
import cv2
import copy
import pandas as pd
import numpy as np
import numpy.matlib as matlib
import matplotlib.pyplot as plt
import multiprocessing
import os
import pickle
import PIL
import PIL.ImageOps
import math
import natsort
import glob
import re
import random
import sys
import scipy
import shutil
import time
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
from IPython import display
from IPython.core.debugger import Tracer
from joblib import Parallel, delayed
from matplotlib.pyplot import figure
from PIL import Image
from scipy import misc
from scipy import signal
from scipy.io import loadmat
from scipy.io import savemat
from sklearn import linear_model
from sklearn.utils import shuffle
from skimage.util import view_as_windows as viewW
from sklearn.neighbors import NearestNeighbors
from skimage.measure import compare_ssim
from skimage import filters
from sobol import *
from tqdm.auto import tqdm
#==================================================================


In [ ]:

#General information regarding samples, used for testing and best C value determination
class Sample:
    def __init__(self, name, images, massRanges, maskObject, mzWeights, resultsPath):
        self.name = name
        self.images = images
        self.massRanges = massRanges
        self.maskObject = maskObject
        self.mzWeights = mzWeights
        self.measuredImages = []
        for rangeNum in range(0,len(massRanges)):
            self.measuredImages.append(np.zeros([maskObject.width, maskObject.height]))
        self.resultsPath = resultsPath

#Sample information for training the regression model
class trainingSample:
    def __init__(self, name, images, maskObject, massRanges, measurementPerc, polyFeatures, reconImage, orderForRD, RD):
        self.name = name
        self.images = images
        self.maskObject = maskObject
        self.massRanges = massRanges
        self.measurementPerc = measurementPerc
        self.polyFeatures = polyFeatures
        self.reconImage = reconImage
        self.orderForRD = orderForRD
        self.RD = RD

class SLADSModel:
    def __init__(self, massRange, theta, cValue):
        self.massRange = massRange
        self.theta = theta
        self.cValue = cValue

class Result():
    def __init__(self, info, sample, avgImage, simulationFlag, animationFlag):
        self.info = info
        self.sample = sample
        self.avgImage = avgImage
        self.simulationFlag = simulationFlag
        self.animationFlag = animationFlag
        self.reconImages = []
        self.masks = []
        self.ERDValueNPs = []
        self.TDList = []
        self.MSEList = []
        self.SSIMList = []
        self.percMeasuredList = []
        
    def update(self, percMeasured, reconImage, maskObject, ERDValuesNP, iterNum, completedRunFlag):

        #Save the model development
        self.reconImages.append(reconImage)
        self.masks.append(maskObject.mask.copy())
        self.ERDValueNPs.append(ERDValuesNP.copy())

        if self.simulationFlag:
            
            #Find statistics of interest
            difference = np.sum(computeDifference(self.avgImage, reconImage, self.info.imageType))
            TD = difference/maskObject.area
            MSE = (np.sum((reconImage.astype("float") - self.avgImage.astype("float")) ** 2))/(float(maskObject.area))
            SSIM = compare_ssim(reconImage.astype("float"), self.avgImage.astype("float"))

            #Save them for each timestep
            self.TDList.append(TD)
            self.MSEList.append(MSE)
            self.SSIMList.append(SSIM)
            self.percMeasuredList.append(percMeasured)
        
        #If an animation will be produced and the run has completed
        if self.animationFlag and completedRunFlag:
            
            #Setup directory addresses
            dir_Animations = self.sample.resultsPath+ 'Animations/'
            dir_AnimationVideos = dir_Animations + 'Videos/'
            dir_AnimationFrames = dir_Animations + self.sample.name + '/'

            if os.path.exists(dir_AnimationFrames): shutil.rmtree(dir_AnimationFrames)    
            os.makedirs(dir_AnimationFrames)
            
        #If an animation should be produced and the run has completed for a simulation
        if self.simulationFlag and self.animationFlag and completedRunFlag:
            
            #Normalize the ERDValuesNP arrays across entire series of images
            ERDValuesNPMax = 0
            for ERDValuesNP in self.ERDValueNPs:
                if np.max(ERDValuesNP) > ERDValuesNPMax:
                    ERDValuesNPMax = np.max(ERDValuesNP)
            for ERDValuesNP in self.ERDValueNPs: ERDValuesNP *= (255.0/ERDValuesNPMax)
             
            #Generate each of the frames
            for i in range(0, len(self.masks)):
                saveLocation = dir_AnimationFrames + 'original_' + self.sample.name + '_iter_' + str(i+1) + '_perc_' + str(self.percMeasuredList[i]) + '.png'
                
                font = {'size' : 18}
                plt.rc('font', **font)
                f = plt.figure(figsize=(15,10))

                f.subplots_adjust(top = 0.85)
                f.subplots_adjust(wspace=0.15, hspace=0.2)
                plt.suptitle("Percent Sampled: %.2f, Measurement Iteration: %.0f\nSSIM: %.2f" % (self.percMeasuredList[i], i+1, self.SSIMList[i]), fontsize=20, fontweight='bold', y = 0.95)
                sub = f.add_subplot(2,2,1)
                sub.imshow(self.avgImage * 255.0/self.avgImage.max(), cmap='hot')
                sub.set_title('Ground-Truth')

                sub = f.add_subplot(2,2,2)
                sub.imshow(self.reconImages[i] * 255.0/self.reconImages[i].max(), cmap='hot')
                sub.set_title('Reconstructed Image')

                sub = f.add_subplot(2,2,3)
                sub.imshow(self.masks[i], cmap='gray')
                sub.set_title('Sampled Mask')

                sub = f.add_subplot(2,2,4)
                im = sub.imshow(self.ERDValueNPs[i], cmap='viridis', vmin=0, vmax=255)
                sub.set_title('ERD Values')
                cbar = f.colorbar(im, ax=sub, orientation='vertical', pad=0.01)
                
                plt.savefig(saveLocation, bbox_inches='tight')
                plt.close()
                
                #Stretched variant
                saveLocation = dir_AnimationFrames + 'stretched_' + self.sample.name + '_iter_' + str(i+1) + '_perc_' + str(self.percMeasuredList[i]) + '.png'
                
                font = {'size' : 18}
                plt.rc('font', **font)
                f = plt.figure(figsize=(15,15))

                f.subplots_adjust(top = 0.85)
                f.subplots_adjust(wspace=0.15, hspace=0.2)
                plt.suptitle("Percent Sampled: %.2f, Measurement Iteration: %.0f\nSSIM: %.2f" % (self.percMeasuredList[i], i+1, self.SSIMList[i]), fontsize=20, fontweight='bold', y = 0.95)
                sub = f.add_subplot(2,2,1)
                sub.imshow(self.avgImage * 255.0/self.avgImage.max(), cmap='hot', aspect='auto')
                sub.set_title('Ground-Truth')

                sub = f.add_subplot(2,2,2)
                sub.imshow(self.reconImages[i] * 255.0/self.reconImages[i].max(), cmap='hot', aspect='auto')
                sub.set_title('Reconstructed Image')

                sub = f.add_subplot(2,2,3)
                sub.imshow(self.masks[i], cmap='gray', aspect='auto')
                sub.set_title('Sampled Mask')

                sub = f.add_subplot(2,2,4)
                im = sub.imshow(self.ERDValueNPs[i], cmap='viridis', vmin=0, vmax=255, aspect='auto')
                sub.set_title('ERD Values')
                cbar = f.colorbar(im, ax=sub, orientation='vertical', pad=0.01)
                
                plt.savefig(saveLocation, bbox_inches='tight')
                plt.close()
            
            #Form the full animation
            dataFileNames = natsort.natsorted(glob.glob(dir_AnimationFrames + 'original_*.png'))
            height, width, layers = cv2.imread(dataFileNames[0]).shape
            animation = cv2.VideoWriter(dir_AnimationVideos + 'original' + self.sample.name + '.avi', cv2.VideoWriter_fourcc(*'MJPG'), 2, (width, height))
            for specFileName in dataFileNames: animation.write(cv2.imread(specFileName))
            animation.release()
            animation = None
            
            dataFileNames = natsort.natsorted(glob.glob(dir_AnimationFrames + 'stretched_*.png'))
            height, width, layers = cv2.imread(dataFileNames[0]).shape
            animation = cv2.VideoWriter(dir_AnimationVideos + 'stretched_' + self.sample.name + '.avi', cv2.VideoWriter_fourcc(*'MJPG'), 2, (width, height))
            for specFileName in dataFileNames: animation.write(cv2.imread(specFileName))
            animation.release()
            animation = None

class Info:
    def __init__(self, reconMethod, featReconMethod, neighborWeightsPower, numNeighbors, filterType, featDistCutoff, resolution, imageType, shouldUpdate, minRadius, maxRadius, incRadius, numNbrs):
        self.reconMethod = reconMethod
        self.featReconMethod = featReconMethod
        self.neighborWeightsPower = neighborWeightsPower
        self.numNeighbors = numNeighbors
        self.filterType = filterType
        self.featDistCutoff = featDistCutoff
        self.resolution = resolution
        self.imageType = imageType
        self.shouldUpdate = shouldUpdate
        self.minRadius = minRadius
        self.maxRadius = maxRadius
        self.incRadius = incRadius
        self.numNbrs = numNbrs

#Storage location for the stopping parameters
class StopCondParams:
    def __init__(self, area, threshold, JforGradient, minPercentage, maxPercentage):
        if area<512**2+1:
            self.beta = 0.001*(((18-math.log(area,2))/2)+1)
        else:
            self.beta = 0.001/(((math.log(area,2)-18)/2)+1)
        self.threshold = threshold
        self.JforGradient = JforGradient
        self.minPercentage = minPercentage
        self.maxPercentage = maxPercentage

#Each sample needs a mask object
class MaskObject():
    def __init__(self, width, height, measurementPercs):
        self.width = width
        self.height = height
        self.area = width*height
        self.initialMaskPts = []
        self.percMasks = []
        self.measuredIdxs = []
        self.unMeasuredIdxs = []
        self.initialMeasuredIdxs = []
        self.initialUnMeasuredIdxs = []
        self.unMeasuredIdxsList = []
        self.measuredIdxsList = []
        
        #Generate a list of arrays contianing the x,y points that need to be scanned
        self.linesToScan = []
        for rowNum in np.arange(0,height,1):
            line = []
            for columnNum in np.arange(0, width, 1):
                line.append(tuple([rowNum, columnNum]))
            self.linesToScan.append(line)

        #Generate the initial set of linesToScan
        
        #50%
        self.originalLinesToScan = copy.copy(self.linesToScan)
        self.initialMask = np.zeros([height, width])
        lineIndex = int(height*0.5)
        for pt in self.linesToScan[lineIndex]: 
            self.initialMask[tuple(pt)] = 1
            self.initialMaskPts.append(pt)
        self.delLine(lineIndex)
        
        #25%
        lineIndex = int(height*0.25)
        for pt in self.linesToScan[lineIndex]: 
            self.initialMask[tuple(pt)] = 1
            self.initialMaskPts.append(pt)
        self.delLine(lineIndex)
        
        #75%
        lineIndex = int(height*0.75)
        for pt in self.linesToScan[lineIndex]: 
            self.initialMask[tuple(pt)] = 1
            self.initialMaskPts.append(pt)
        self.delLine(lineIndex)
        
        self.initialLinesToScan = copy.copy(self.linesToScan)
        self.initialMeasuredIdxs = np.transpose(np.where(self.initialMask == 1))
        self.initialUnMeasuredIdxs = np.transpose(np.where(self.initialMask == 0))
        
        #Create random initial percentage masks using point measurements instead of full lines
        for measurementPerc in measurementPercs:
            self.mask = np.zeros([height, width])
            self.mask = np.random.rand(height, width) < (measurementPerc/100)
            self.percMasks.append(self.mask)
            self.measuredIdxsList.append(np.transpose(np.where(self.mask == 1)))
            self.unMeasuredIdxsList.append(np.transpose(np.where(self.mask == 0)))
        
        #Create random line masks, using full lines
#        for measurementPerc in measurementPercs:
#            self.mask = copy.copy(self.initialMask)
#            self.linesToScan = copy.copy(self.initialLinesToScan)
#            self.measuredIdxs = copy.copy(self.initialMeasuredIdxs)
#            self.unMeasuredIdxs = copy.copy(self.initialUnMeasuredIdxs)
#            while (np.sum(self.mask)/self.area)*100 < measurementPerc:
#                lineIndex = int((np.random.rand(1)[0]*len(self.linesToScan)))
#                for pt in self.linesToScan[lineIndex]: self.mask[tuple(pt)] = 1
#                self.delLine(lineIndex)
#            self.percMasks.append(self.mask)
#            self.measuredIdxsList.append(np.transpose(np.where(self.mask == 1)))
#            self.unMeasuredIdxsList.append(np.transpose(np.where(self.mask == 0)))
    
    #Update the mask given a set of new measurement locations
    def update(self, newIdxs):
        for pt in newIdxs: self.mask[tuple(pt)] = 1
        self.measuredIdxs = np.transpose(np.where(self.mask == 1))
        self.unMeasuredIdxs = np.transpose(np.where(self.mask == 0))
    
    #Reset the training sample's mask and linesToScan to nothing having been scanned
    def reset(self):
        self.mask = np.zeros([self.height, self.width])
        self.linesToScan = copy.copy(self.originalLinesToScan)
        self.measuredIdxs = np.transpose(np.where(self.mask == 1))
        self.unMeasuredIdxs = np.transpose(np.where(self.mask == 0))
            
    def delLine(self, index):
        self.linesToScan = np.delete(self.linesToScan, index, 0)
        
    def delPoints(self, pts):
        for i in range(0,len(self.linesToScan)):
            indexes = []
            for pt in pts:
                indexes.append([i for i, j in enumerate(self.linesToScan[i]) if j == pt])
            indexes = [x for x in np.asarray(indexes).flatten().tolist() if x != []]
            if len(indexes) > 0:
                self.linesToScan[i] = np.delete(self.linesToScan[i], indexes,0).tolist()

def runSLADS(info, sample, maskObject, theta, stopPerc, simulationFlag, trainPlotFlag, animationFlag):
    
    #Reinitialize the mask state as blank
    maskObject.reset()
    
    #Has the stopping condition been met yet
    completedRunFlag = False
    
    #Current iteration
    iterNum = 1
    
    #Assume variable Classify=='N' (artifact from original implementation)
    
    #Perform weighted averaging for the ground-truth image
    if simulationFlag:
        npImages = []
        for image in sample.images: npImages.append(np.asarray(image))
        avgImage = np.average(np.asarray(npImages), axis=0, weights=sample.mzWeights)

    
    #Initialize stopping condition object
    stopCondParams = StopCondParams(maskObject.area, 0, 50, 2, stopPerc)
    
    #Determine stoppingCondition function value
    stopCondFuncVal = np.zeros((int((maskObject.area)*(stopCondParams.maxPercentage)/100)+10,2))
    
    #Perform the initial measurements
    sample, maskObject = performMeasurements(sample, maskObject, maskObject.initialMeasuredIdxs, simulationFlag)
    
    #Perform initial reconstruction and ERD calculation
    reconImage, reconValues, ERDValues, ERDValuesNP, reconImageList, reconValuesList, ERDValueList = avgReconAndERD(sample, info, iterNum, maskObject, theta, reconImage=None, reconValues=None, ERDValues=None, ERDValuesNP=None, newIdxs=None, maxIdxsVect=None, reconImageList=None, reconValuesList=None, ERDValueList=None)

    #Determine percentage pixels measured initially
    percMeasured = (np.sum(maskObject.mask)/maskObject.area)*100

    #Retrieve the averaged image's measured values
    measuredValues = np.asarray(avgImage)[maskObject.mask == 1]
    
    #Check for completion state here just in case, prior to loop!
    completedRunFlag = checkStopCondFuncThreshold(stopCondParams, stopCondFuncVal, maskObject, measuredValues, iterNum)
    
    #Additional stopping condition for if there are no more linesToScan
    if len(maskObject.linesToScan) == 0: completedRunFlag = True
    
    
    #Initialize a result object
    result = Result(info, sample, avgImage, simulationFlag, animationFlag)
    result.update(percMeasured, reconImage, maskObject, ERDValuesNP, iterNum, completedRunFlag)
    
    #Until the stopping criteria has been met
    with tqdm(total = float(stopPerc), desc = '% Sampled', leave = True) as pbar:
        
        #Initialize progress bar state according to % measured
        pbar.n = round(percMeasured,2)
        pbar.refresh()
        
        #Until the program has completed
        while not completedRunFlag:
            
            #Step the iteration counter
            iterNum += 1
            
            #Make a duplicate of the ReconValues for stop condition gradient test
            oldReconValues = reconValues.copy()
            
            #Find next measurement locations
            maskObject, newIdxs, maxIdxsVect = findNewMeasurementIdxs(info, maskObject, measuredValues, theta, reconValues, reconImage, ERDValues, ERDValuesNP)
            
            #Perform measurements
            sample, maskObject = performMeasurements(sample, maskObject, newIdxs, simulationFlag)
            
            #Perform reconstruction and ERD calculation
            reconImage, reconValues, ERDValues, ERDValuesNP, reconImageList, reconValuesList, ERDValueList = avgReconAndERD(sample, info, iterNum, maskObject, theta, reconImage, reconValues, ERDValues, ERDValuesNP, newIdxs, maxIdxsVect, reconImageList, reconValuesList, ERDValueList)
            
            #Update the percentage of pixels that have beene measured
            percMeasured = (np.sum(maskObject.mask)/maskObject.area)*100
            
            #Evaluate the stop condition value 
            stopCondFuncVal = computeStopCondFuncVal(oldReconValues, measuredValues, stopCondParams, info, stopCondFuncVal, maxIdxsVect, iterNum, maskObject)
            
            #Retrieve the averaged image's measured values
            measuredValues = np.asarray(avgImage)[maskObject.mask == 1]
            
            #Check the stopping condition
            completedRunFlag = checkStopCondFuncThreshold(stopCondParams, stopCondFuncVal, maskObject, measuredValues, iterNum)
            
            #Additional stopping condition for if there are no more linesToScan
            if len(maskObject.linesToScan) == 0: completedRunFlag = True
            
            #Store information to the resultsObject 
            result.update(percMeasured, reconImage, maskObject, ERDValuesNP, iterNum, completedRunFlag)

            #Update the progress bar
            pbar.n = round(percMeasured,2)
            pbar.refresh()
    
    return result

def findNewMeasurementIdxs(info, maskObject, measuredValues, theta, reconValues, reconImage, ERDValues, ERDValuesNP):
    newIdxs = []
    maxIdxsVect = []
    
    #Assuming manual angle selection; sum ERD for all lines
    lineERDValues = []
    for line in maskObject.linesToScan:
        lineERDValues.append(np.nansum(ERDValuesNP[tuple([x[0] for x in line]), tuple([y[1] for y in line])]))
    
    lineToScanIdx = np.nanargmax(lineERDValues)
    
    #Set threshold for what ERD Values are worth scanning the locations of
    threshold = 0
    #threshold = np.mean(ERDValuesNP[np.where((ERDValuesNP > 0))])
    
    #Determine start and end points to measure between on the line with greatest sum ERD
    startIDXFound = False
    endIDXFound = False  
    
    #filteredIdxs = np.argwhere(ERDValuesNP[lineToScanIdx] > threshold)
    #if len(filteredIdxs) > 0:
    #    startIDX, endIDX = min(filteredIdxs), max(filteredIdxs)
    #    newIdxs = filteredIdxs[startIDX:endIdx]
    #else:
    #     
    
    if lineERDValues[lineToScanIdx] > 0:
        for idx in maskObject.linesToScan[lineToScanIdx]:
            if ERDValuesNP[tuple(idx)] > threshold and not startIDXFound:
                startIDX = idx
                startIDXFound = True
            if ERDValuesNP[tuple(idx)] > threshold:
                endIDX = idx

        #Add points between start and end idxs to the list if they were determined
        if (startIDXFound):
            for idx in maskObject.linesToScan[lineToScanIdx]:
                if idx[0] >= startIDX[0] and idx[1] >= startIDX[1] and idx[0] <= endIDX[0] and idx[1] <= endIDX[1]:
                    newIdxs.append(idx)
        else:
            #No location was found over the threshold, just scan the full line
            newIdxs = maskObject.linesToScan[lineToScanIdx]
    else:
        #Just choose a line to scan...
        newIdxs = maskObject.linesToScan[lineToScanIdx]

    #Remove the line selected from further consideration
    maskObject.delLine(lineToScanIdx)

    ptArray = np.asarray(newIdxs).tolist()
    for k in range(0, len(newIdxs)):
        pt = ptArray[k]
        if pt in maskObject.unMeasuredIdxs.tolist():
            maxIdxsVect.append(maskObject.unMeasuredIdxs.tolist().index(pt))

    newIdxs = np.asarray(newIdxs) #Actual coordinates to measure next
    maxIdxsVect = np.asarray(maxIdxsVect) #Positions of those coordinates within the unmeasured listing

    return maskObject, newIdxs, maxIdxsVect

def avgReconAndERD(sample, info, iterNum, maskObject, theta, reconImage, reconValues, ERDValues, ERDValuesNP, newIdxs, maxIdxsVect, reconImageList, reconValuesList, ERDValueList):
    
    #Find neighbor information
    neighborIndices, neighborWeights, neighborDistances = findNeighbors(info, maskObject.measuredIdxs, maskObject.unMeasuredIdxs)
    
    if iterNum >= 1:
        #For all of the images in the sample, find the recon Images, Values, and ERD Values
        reconImageList = []
        reconValuesList = []
        ERDValueList = []
        for image in sample.measuredImages:
            
            #Retrieve the measured values
            measuredValues = np.asarray(image)[maskObject.mask == 1]
            
            #Find neighborhood values
            neighborValues = findNeighborValues(measuredValues, neighborIndices)

            #Compute reconstructions
            reconValues, reconImage = computeRecons(info, maskObject, maskObject.unMeasuredIdxs, maskObject.measuredIdxs, neighborValues, neighborWeights, measuredValues)

            #Compute full ERD Values
            ERDValues = computeFullERD(info, maskObject, measuredValues, reconValues, reconImage, theta, neighborValues, neighborWeights, neighborDistances)
            
            #Store results
            reconImageList.append(reconImage)
            reconValuesList.append(reconValues)
            ERDValueList.append(ERDValues)

    else:
        sys.exit('Error! - Turned off the computeUpdateERD function!')
        #Perform updates to the individual reconValues reconImages and ERDValues
        for imageNum in range(0,len(sample.measuredImages)):
            #Retrieve the measured values
            measuredValues = np.asarray(sample.measuredImages[imageNum])[maskObject.mask == 1]
            
            #Update the ERD Values
            reconValuesList[imageNum], reconImageList[imageNum], ERDValueList[imageNum] = computeUpdateERD(maskObject, measuredValues, theta, info, newIdxs, reconValuesList[imageNum], reconImageList[imageNum], ERDValueList[imageNum], maxIdxsVect)

    #Perform weighted averaging
    reconImage = np.average(np.asarray(reconImageList), axis=0, weights=sample.mzWeights)
    reconValues = np.average(np.asarray(reconValuesList), axis=0, weights=sample.mzWeights)
    ERDValues = np.average(np.asarray(ERDValueList), axis=0, weights=sample.mzWeights)

    #Convert ERDValues from 1D to 2D
    ERDValuesNP = makeERDArray(ERDValues, maskObject, reconImage)

    return reconImage, reconValues, ERDValues, ERDValuesNP, reconImageList, reconValuesList, ERDValueList

def computeFullERD(info, maskObject, measuredValues, reconValues, reconImage, theta, neighborValues, neighborWeights, neighborDistances):

    # Compute features
    polyFeatures = computeFeatures(maskObject.unMeasuredIdxs, maskObject.area, neighborValues, neighborWeights, neighborDistances, info, reconValues, reconImage)
    
    # Compute ERD
    ERDValues = polyFeatures.dot(theta)
    
    return(ERDValues)

def computeUpdateERD(maskObject, measuredValues, theta, info, newIdxs, reconValues, reconImage, ERDValues, maxIdxsVect):  

    ERDValues = np.delete(ERDValues,(maxIdxsVect))
    
    reconValues = np.delete(reconValues,(maxIdxsVect))
    
    suggestedRadius = int(np.sqrt((1/np.pi)*(maskObject.area*info.numNbrs/np.shape(measuredValues)[0])))
    
    updateRadiusTemp = np.max([suggestedRadius, info.minRadius]);
    
    updateRadius = int(np.min([info.maxRadius, updateRadiusTemp]));

    updateRadiusMat = np.zeros((maskObject.height, maskObject.width))
    
    while(True):
        for newIdxNum in range(0,len(newIdxs)):
            #height and width might be reversed
            updateRadiusMat[max(newIdxs[newIdxNum][0]-updateRadius,0):min(newIdxs[newIdxNum][0]+updateRadius,maskObject.height)][:,max(newIdxs[newIdxNum][1]-updateRadius,0):min(newIdxs[newIdxNum][1]+updateRadius,maskObject.width)]=1
        updateIdxs = np.where(updateRadiusMat[maskObject.mask==0]==1)
        smallUnMeasuredIdxs = np.transpose(np.where(np.logical_and(maskObject.mask==0, updateRadiusMat==1)))
        if smallUnMeasuredIdxs.size == 0:
            updateRadius = int(updateRadius*info.incRadius)
        else:
            break
    
    #Determine neighborhood information
    smallNeighborIndicies, smallNeighborWeights, smallNeighborDistances = findNeighbors(info, maskObject.measuredIdxs, smallUnMeasuredIdxs)
    
    smallNeighborValues = findNeighborValues(measuredValues, smallNeighborIndicies)
    
    # Perform reconstruction
    smallReconValues = computeWeightedMRecons(smallNeighborValues, smallNeighborWeights, info)
    
    reconImage[(np.logical_and(maskObject.mask==0, updateRadiusMat==1))] = smallReconValues
    reconImage[maskObject.measuredIdxs[:,0], maskObject.measuredIdxs[:,1]] = measuredValues

    # Compute features
    smallPolyFeatures = computeFeatures(smallUnMeasuredIdxs, maskObject.area, smallNeighborValues, smallNeighborWeights, smallNeighborDistances, info, smallReconValues, reconImage)

    # Compute ERD
    smallERDValues = smallPolyFeatures.dot(theta)
    reconValues[updateIdxs[0]] = smallReconValues
    ERDValues[updateIdxs] = smallERDValues
    
    return(reconValues, reconImage, ERDValues)

def makeERDArray(ERDValues, maskObject, reconImage):
    #Rearrange ERD values into array; those that have already been measured have 0 ERD
    ERDValuesNP = np.zeros([maskObject.height, maskObject.width])
    
    #Copy over ERD values for unmeasured points
    for i in range(0, len(maskObject.unMeasuredIdxs)): ERDValuesNP[maskObject.unMeasuredIdxs[i][0], maskObject.unMeasuredIdxs[i][1]] = ERDValues[i]

    #Remove values that are less than those already scanned (0 ERD)
    ERDValuesNP[np.where((ERDValuesNP < 0))] = 0
    
    #The following masks the ERD Values array by the reconstruction image
    #Dr. Ye has requested that this masking procedure, which limits scanning by the determined structure
    #be deactivated
    #ERDValuesNP[np.where((reconImage < np.std(reconImage)))] = 0
    
    return ERDValuesNP

def checkStopCondFuncThreshold(stopCondParams, StopCondFuncVal, maskObject, measuredValues, iterNum):
    
    if stopCondParams.threshold == 0:
        if np.shape(measuredValues)[0] >= round(maskObject.area*stopCondParams.maxPercentage/100):
            return True
        else:
            return False
    else:
        if np.shape(measuredValues)[0] >= round(maskObject.area*stopCondParams.maxPercentage/100):
            return True
        else:
            if np.logical_and(((maskObject.area)*stopCondParams.minPercentage/100)<np.shape(measuredValues)[0], stopCondFuncVal[iterNum,0]<stopCondParams.threshold):
                gradStopCondFunc = np.mean(stopCondFuncVal[iterNum,0]-stopCondFuncVal[iterNum-stopCondParams.JforGradient:iterNum-1,0])
                if gradStopCondFunc < 0:
                    return True
            else:
                return False

def computeStopCondFuncVal(reconValues, measuredValues, stopCondParams, info, stopCondFuncVal, maxIdxsVect, iterNum, maskObject):
    
    #Calculate the difference in values between the previous reconstruction values against the measured values
    diff = 0
    for i in range(0, len(maxIdxsVect)):
        a = reconValues[maxIdxsVect[i]]
        b = measuredValues[len(measuredValues)-len(maxIdxsVect)+i]
        diff = computeDifference(a, b, info.imageType) + diff
    diff = diff/len(measuredValues)
    
    if iterNum == 1:
        stopCondFuncVal[iterNum,0] = stopCondParams.beta*diff
    else:
        stopCondFuncVal[iterNum,0] = ((1-stopCondParams.beta)*stopCondFuncVal[iterNum-1,0] + stopCondParams.beta*diff)
    
    stopCondFuncVal[iterNum,1] = np.shape(measuredValues)[0]
    
    return stopCondFuncVal               

def findNeighbors(info, measuredIdxs, unMeasuredIdxs):
    
    neigh = NearestNeighbors(n_neighbors=info.numNeighbors)
    neigh.fit(measuredIdxs)
    neighborDistances, neighborIndices = neigh.kneighbors(unMeasuredIdxs)
    neighborDistances = neighborDistances*info.resolution
    unNormNeighborWeights = 1/np.power(neighborDistances, info.neighborWeightsPower)
    sumOverRow = (np.sum(unNormNeighborWeights, axis=1))
    neighborWeights = unNormNeighborWeights/sumOverRow[:, np.newaxis]
    
    return neighborIndices, neighborWeights, neighborDistances

def findNeighborValues(measuredValues, neighborIndices):
    return measuredValues[neighborIndices]

def computeRecons(info, maskObject, unMeasuredIdxs, measuredIdxs, neighborValues, neighborWeights, measuredValues):
    reconValues = computeWeightedMRecons(neighborValues, neighborWeights, info)
    reconImage = np.zeros((maskObject.height, maskObject.width))
    reconImage[unMeasuredIdxs[:,0], unMeasuredIdxs[:,1]] = reconValues
    reconImage[measuredIdxs[:,0], measuredIdxs[:,1]] = measuredValues
    return(reconValues, reconImage)

def computeWeightedMRecons(neighborValues, neighborWeights, info):
    if info.featReconMethod=='DWM':
        classLabels = np.unique(neighborValues)
        classWeightSums = np.zeros((np.shape(neighborWeights)[0], np.shape(classLabels)[0]))
        for i in range(0,np.shape(classLabels)[0]):
            tempFeats = np.zeros((np.shape(neighborWeights)[0], np.shape(neighborWeights)[1]))
            np.copyto(tempFeats, neighborWeights)
            tempFeats[neighborValues != classLabels[i]]=0
            classWeightSums[:,i] = np.sum(tempFeats, axis=1)
        reconValues = classLabels[np.argmax(classWeightSums, axis=1)]
    elif info.featReconMethod=='CWM':
        reconValues = np.sum(neighborValues*neighborWeights, axis=1)
    return reconValues

def computeFeatures(unMeasuredIdxs, area, neighborValues, neighborWeights, neighborDistances, info, reconValues, reconImage):
    feature = np.zeros((np.shape(unMeasuredIdxs)[0],6))

    # Compute std div features
    diffVect = computeDifference(neighborValues, np.transpose(np.matlib.repmat(reconValues, np.shape(neighborValues)[1],1)), info.imageType)
    feature[:,0] = np.sum(neighborWeights*diffVect, axis=1)
    feature[:,1] = np.sqrt((1/info.numNeighbors)*np.sum(np.power(diffVect,2),axis=1))
    
    # Compute distance/density features
    cutoffDist = np.ceil(np.sqrt((info.featDistCutoff/100)*(area/np.pi)))
    feature[:,2] = neighborDistances[:,0]
    neighborsInCircle= np.sum(neighborDistances <= cutoffDist, axis=1)
    feature[:,3] = (1+(np.pi*(np.power(cutoffDist, 2))))/(1+np.sum(neighborDistances <= cutoffDist, axis=1))

    # Compute gradient features
    gradientImageX, gradientImageY = np.gradient(reconImage)

    #Assume continuous features
    gradientImageX = abs(gradientImageX)
    gradientImageY = abs(gradientImageY)
    feature[:,4] = gradientImageY[unMeasuredIdxs[:,0], unMeasuredIdxs[:,1]]
    feature[:,5] = gradientImageX[unMeasuredIdxs[:,0], unMeasuredIdxs[:,1]]
    
    #Compute polyfeatures
    polyFeatures = np.hstack([np.ones((np.shape(feature)[0],1)), feature])
    for i in range(0, np.shape(feature)[1]):
        for j in range(i, np.shape(feature)[1]):
            temp = feature[:,i]*feature[:,j]
            polyFeatures = np.column_stack([polyFeatures, feature[:,i]*feature[:,j]])
    return polyFeatures

def computeDifference(array1, array2, imageType):
    if imageType == 'C':
        return abs(array1-array2)
    elif imageType == 'D':
        difference = array1 != array2
        return difference.astype(float)
    else:
        sys.exit('Error! - Unexpected imageType declared')
        return 0
    
def cls(): #Clear console screen
    os.system('cls' if os.name=='nt' else 'clear')   

def cAreaCalc(c, neighborDistances, unMeasuredIdxs, windowSize, imgAsBlocksOnlyUnmeasured):
    temp = np.zeros((windowSize*windowSize,len(unMeasuredIdxs)))
    sigma = neighborDistances[:,0]/c
    area = []
    for index in tqdm(range(0,len(unMeasuredIdxs)), desc = 'Kernel'):     
            ax = np.linspace(-(windowSize - 1) / 2., (windowSize - 1) / 2., windowSize)
            xx, yy = np.meshgrid(ax, ax)
            area.append(imgAsBlocksOnlyUnmeasured[:,index]*np.ravel(np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sigma[index]))))

    for i in range (0,len(unMeasuredIdxs)): temp[:,i] = area[i]
    
    #Determine how much "area of uncertainty" is possibly removed for a c value
    return np.sum(temp, axis=0)
    
def generateGaussianKernel(sigma, windowSize):
    GK1 = signal.gaussian(windowSize, std=sigma)
    return np.outer(GK1, GK1).flatten()

def percResults(results, perc_testingResults, precision):
    percents = np.arange(min(np.hstack(perc_testingResults)),max(np.hstack(perc_testingResults))+precision, precision)
    averages = []
    finalPercents = []
    for percent in percents:
        values = []
        for i in range(0,len(results)):
            percList = np.array(perc_testingResults[i])
            idx = np.argmin(np.abs(np.asarray(percList)-percent))
            values.append(results[i][idx])
        averageValue = np.mean(values)
        if len(averages) == 0 or averageValue != averages[len(averages)-1]:
            averages.append(np.mean(values))
            finalPercents.append(percent)
    return finalPercents, averages

def testing_parhelper(testingSampleFolder, info, bestTheta, stopPerc, animationGeneration):
    dataSampleName = os.path.basename(testingSampleFolder)
    
    #Obtain testing images
    images = []
    for imageFileName in natsort.natsorted(glob.glob(testingSampleFolder + '/*.' + 'csv'), reverse=False):
        images.append(np.nan_to_num(np.loadtxt(imageFileName, delimiter=',')))
        #images.append(Image.fromarray((libtiff.TIFF.open(imageFileName).read_image()*255.0).astype('uint8')).convert('L') )
    
    #Read in the width and height; when RAW files are supported
    #width, height = pd.read_csv(testingSampleFolder+'/dimensions.csv', sep=',', header=None).values[0].tolist()
    height, width = images[0].shape
    
    #Create a new maskObject
    maskObject = MaskObject(width, height, measurementPercs=[])
    
    #How should the mz ranges be weighted (all equal for now)
    mzWeights = np.ones(len(desiredMassRanges))/len(desiredMassRanges)
    
    #Define information as a new Sample object
    testingSample = Sample(dataSampleName, images, desiredMassRanges, maskObject, mzWeights, dir_TestingResults)
    
    #Run SLADS till its stopping condition has been met
    return runSLADS(info, testingSample, maskObject, bestTheta, stopPerc, simulationFlag=True, trainPlotFlag=False, animationFlag=animationGeneration)
    
def performMeasurements(sample, maskObject, newIdxs, simulationFlag):
    #Update the maskObject according to the newIdxs
    maskObject.update(newIdxs)
    
    #If performing training
    if simulationFlag:
        #Obtain values from the stored image information
        for imageNum in range(0,len(sample.measuredImages)):
            temp = np.asarray(sample.images[imageNum]).copy()
            temp[maskObject.mask == 0] = 0
            sample.measuredImages[imageNum] = temp.copy()
    else:
        sys.exit('ERROR! - Runtime variation for perform measurements has not yet been implemented!')
        #For pt in newIdxs
        #Read in the designated locations tuple(pt) to a .RAW file
        #Perform discretizations for sample.mzRanges
        #For each of the discretizations to be stored
        #for imageNum in range(0,len(sample.measuredImages)):
            #sample.measuredImages[imageNum][tuple(pt)] = correspondingValue
    return sample, maskObject

#==================================================================


In [ ]:
#ESSENTIALLY NON-ESSENTIAL PARAMETERS
#==================================================================
#Running in a console/True, notebook/False
consoleRunning = False

#ESSENTIALLY NON-ESSENTIAL SETUP
#==================================================================
#Clear the screen
cls()

#Determine console size if applicable
if consoleRunning:
    consoleRows, consoleColumns = os.popen('stty size', 'r').read().split()
else:
    consoleRows, consoleColumns = 40, 40

In [ ]:
#==================================================================
#MAIN PROGRAM
#==================================================================

#GENERAL PARAMETERS: L-01
#==================================================================
#Is training of a model to be performed
trainingModel = True

#Is testing of a model to be performed
testingModel = False

#STATIC VARIABLE SETUP
#==================================================================

#What mass ranges should be used for the testing data
desiredMassRanges = [[754.5212, 754.5513],
      [782.5519, 782.5832],
      [804.5358, 804.568],
      [806.5514, 806.5837],
      [808.567, 808.5994],
      [810.5826, 810.6151],
      [828.5353, 828.5685],
      [830.5509, 830.5842],
      [832.5665, 832.5999],
      [856.5661, 856.6003]]

#Sampling percentages for training
measurementPercs = [1,5,10,20,30,40]

# Possible c values for RD approximation
cValues = np.array([1,2,4,8,16,32,64,128])

#Type of Images: D - for discrete (binary) image; C - for continuous
imageType = 'C'

#Symmetric window size for approximate RD summation - 15 for 512x512
windowSize = 27

#Should a stopping threshold be found that corresponds to the best determined c value
findStopThresh = False

#ERD update parameters
shouldUpdate = True
minRadius = 3
maxRadius = 10
incRadius = 1.5
numNbrs = 10

#Initialize the information object for training and testing
#reconMethod, featReconMethod, neighborWeightsPower, numNeighbors, filterType, featDistCutoff, resolution
info = Info('CWM', 'CWM', 2, 10, 'Gaussian', 0.25, 1, imageType, shouldUpdate, minRadius, maxRadius, incRadius, numNbrs)

#Should animations be generated during testing/implementation
animationGeneration = True

#Percent of reduction in distrotion limit for numRandomChoice determination
percOfRD = 50

#Stopping percentage of pixels
stopPerc = 50

#Set the number of available CPU threads
num_threads = multiprocessing.cpu_count()-2 #Leave a couple to spare


In [ ]:
#PATH/DIRECTORY SETUP
#==================================================================
dir_ResultsAndData = '.' + os.path.sep + 'ResultsAndData' + os.path.sep
dir_InputData = dir_ResultsAndData + 'InputData' + os.path.sep

dir_TrainingData = dir_InputData + 'TrainingData' + os.path.sep
dir_TestingData = dir_InputData + 'TestingData' + os.path.sep

dir_Results = dir_ResultsAndData + 'Results' + os.path.sep

dir_TrainingResults = dir_Results + 'TrainingResults' + os.path.sep
dir_TrainingResultsImages = dir_TrainingResults + 'Images' + os.path.sep

dir_TestingResults = dir_Results + 'TestingResults' + os.path.sep
dir_TestingResultsImages = dir_TestingResults + 'Images' + os.path.sep

#Check general directory structure
if not os.path.exists(dir_ResultsAndData):
    sys.exit('Error - dir_ResultsAndData does not exist')

#Input data directories
if not os.path.exists(dir_InputData):
    sys.exit('Error - dir_InputData does not exist')
if not os.path.exists(dir_TrainingData) and trainingModel:
    sys.exit('Error - dir_TrainingData does not exist')
if not os.path.exists(dir_TestingData) and testingModel:
    sys.exit('Error - dir_InputData does not exist')

#Results directories - reset results folders for new runs
if not os.path.exists(dir_Results):
    sys.exit('Error - dir_Results does not exist')
if trainingModel:
    if os.path.exists(dir_TrainingResults):
        shutil.rmtree(dir_TrainingResults)
    os.makedirs(dir_TrainingResults)
    os.makedirs(dir_TrainingResultsImages)    
    
if testingModel:
    if os.path.exists(dir_TestingResults):
        shutil.rmtree(dir_TestingResults)    
    os.makedirs(dir_TestingResults)
    os.makedirs(dir_TrainingResultsImages)

if animationGeneration:
    dir_Animations = dir_TestingResults + 'Animations/'
    dir_AnimationVideos = dir_Animations + 'Videos/'

    if os.path.exists(dir_Animations): shutil.rmtree(dir_Animations)    
    os.makedirs(dir_Animations)
    if os.path.exists(dir_AnimationVideos): shutil.rmtree(dir_AnimationVideos)    
    os.makedirs(dir_AnimationVideos)


In [2]:
print('\n\n' + ('#' * int(consoleColumns)))
print('PERFORMING INTITIAL COMPUTATIONS')
print(('#' * int(consoleColumns)) + '\n')

#For each of the training samples, import their images in order
#Can switch this section to use .RAW file discretization method instead

#Create a set of training samples for each possible c Value
trainingDatabase = []
for cNum in range(0,len(cValues)): trainingDatabase.append([])

trainingSamples = []
sortedTrainingSampleFolders = natsort.natsorted(glob.glob(dir_TrainingData + '/*'), reverse=False)    
for sampleNum in tqdm(range(0,len(sortedTrainingSampleFolders)), desc = 'Training Samples', leave = True):
    trainingSampleFolder = sortedTrainingSampleFolders[sampleNum]
    
    dataSampleName = os.path.basename(trainingSampleFolder)
    images = []
    massRanges = []

    #Import each of the images according to their mz range order
    for imageFileName in natsort.natsorted(glob.glob(trainingSampleFolder + '/*.' + 'csv'), reverse=False):
        images.append(np.nan_to_num(np.loadtxt(imageFileName, delimiter=',')))
        #images.append(Image.fromarray((libtiff.TIFF.open(imageFileName).read_image()*255.0).astype('uint8')).convert('L') )
        massRanges.append([os.path.basename(imageFileName)[2:10], os.path.basename(imageFileName)[11:19]])
    maskObject = MaskObject(images[0].shape[1], images[0].shape[0], measurementPercs)

    #How should the mz ranges be weighted (all equal for now)
    mzWeights = np.ones(len(images))/len(images)
    
    #Append the basic information for each of the provided samples for use in determining best c Value
    trainingSamples.append(Sample(dataSampleName, images, massRanges, maskObject, mzWeights, dir_TrainingResults))
    
    #For each of the measurement percentages, extract features and initial RD values for each image
    for measurementPercNum in tqdm(range(0,len(measurementPercs)), desc = 'Measurement %', leave = True):
        measurementPerc = measurementPercs[measurementPercNum]
        
        #Retreive relevant mask information
        mask = maskObject.percMasks[measurementPercNum]
        measuredIdxs = maskObject.measuredIdxsList[measurementPercNum]
        unMeasuredIdxs = maskObject.unMeasuredIdxsList[measurementPercNum]
        
        #Find neighbor information
        neighborIndices, neighborWeights, neighborDistances = findNeighbors(info, measuredIdxs, unMeasuredIdxs)
        
        #Calculate the sigma values for each possible c
        sigmaValues = []
        for c in cValues: sigmaValues.append(neighborDistances[:,0]/c)
        
        #Flatten 2D mask array to 1D
        maskVect = np.ravel(mask)
        
        #Create a random distribution, depending on # of unmeasured points
        numRandChoices = int((100.0-measurementPercs[measurementPercNum])*percOfRD*maskObject.area/math.pow(10,math.ceil(math.log10(maskObject.area))))

        orderForRD = random.sample(range(0,np.transpose(np.where(mask==0)).shape[0]), numRandChoices) 
        
        #Form reconstructions and determine polyFeatures for each of the images
        reconImageList = []
        polyFeaturesList = []
        for imNum in tqdm(range(0,len(images)), desc = 'Image Features', leave = True):
            image = np.asarray(images[imNum])
            
            #Obtain the measured values
            measuredValues = image[mask==1]
            
            #Find neighborhood values
            neighborValues = findNeighborValues(measuredValues, neighborIndices)
            
            #Compute reconstructions
            reconValues, reconImage = computeRecons(info, maskObject, unMeasuredIdxs, measuredIdxs, neighborValues, neighborWeights, measuredValues)
            
            #Compute features
            polyFeatures = computeFeatures(unMeasuredIdxs, maskObject.area, neighborValues, neighborWeights, neighborDistances, info, reconValues, reconImage)
            
            #Extract random set of the polyFeatures
            polyFeatures = polyFeatures[orderForRD,:]
        
            #Append to set for employment of averaging method
            reconImageList.append(reconImage)
            polyFeaturesList.append(polyFeatures)
        
        #Average the reconstructions according to the mzWeights
        reconImage = np.average(np.asarray(reconImageList), axis=0, weights=mzWeights)
        
        #Average the polyFeatures according to the mzWeights
        polyFeatures = np.average(np.asarray(polyFeaturesList), axis=0, weights=mzWeights)
        
        #Compute the difference between the original and reconstructed images
        RDPP = computeDifference(image, reconImage, info.imageType)

        #Convert differences to int
        RDPP.astype(int)

        #Pad with zeros
        RDPPWithZeros = np.lib.pad(RDPP,(int(np.floor(windowSize/2)),int(np.floor(windowSize/2))), 'constant', constant_values=0)

        #Convert image to an array; faster than im2col implementation
        N = RDPPWithZeros.shape[1] - windowSize + 1
        M = RDPPWithZeros.shape[0] - windowSize + 1
        imgAsBlocks = viewW(RDPPWithZeros, (M,N)).reshape(-1,M*N)[:,::1]

        #Identify the pixels that have not yet been measured
        imgAsBlocksOnlyUnmeasured = imgAsBlocks[:,np.logical_not(maskVect)]

        #Determine RD values for each of the c Values to finalize formation of the training dataset
        temp = np.zeros((windowSize*windowSize, len(orderForRD)))
        for cNum in tqdm(range(0,len(cValues)), desc = 'c Values', leave = True):
            sigma = sigmaValues[cNum]
            c = cValues[cNum]
            #For each of the selected unmeasured points calculate the captured "area"
            for index in tqdm(range(0,len(orderForRD)), desc = 'Gaussian', leave = True):
                temp[:,index] = imgAsBlocksOnlyUnmeasured[:,orderForRD[index]]*generateGaussianKernel(sigma[orderForRD[index]], windowSize)
            
            #Determine RD; the "area of uncertainty" removed
            RD = np.sum(temp, axis=0)
            
            #Add resultant information to the final trainingSamples database for the appropriate c Value
            trainingDatabase[cNum].append(trainingSample(dataSampleName, images, maskObject, massRanges, measurementPerc, polyFeatures, reconImage, orderForRD, RD))
            



NameError: name 'consoleColumns' is not defined

In [ ]:
print('\n\n' + ('#' * int(consoleColumns)))
print('TRAINING MODEL(S)')
print(('#' * int(consoleColumns)) + '\n')
#There exists a single training sample for numCValues*numMeasurementPercs*numTrainingSamples

#Find a SLADS model for each of the c values
trainingModels = []
for cNum in tqdm(range(0,len(cValues)), desc = 'c Values', leave = True): #For each of the proposed c values
    trainingDataset = trainingDatabase[cNum]
    for sampleNum in tqdm(range(0,len(trainingDataset)), desc = 'Training Data', leave = True):
        trainingSample = trainingDataset[sampleNum]      
        if sampleNum == 0: #First loop
            if info.imageType == 'C':
                bigPolyFeatures = trainingSample.polyFeatures
                bigRD = trainingSample.RD
            elif info.imageType == 'D':
                bigPolyFeatures = np.column_stack((trainingSample.polyFeatures[:,0:25], trainingSample.polyFeatures[:,26]))
                bigRD = trainingSample.RD
        else: #Subsequent loops
            if info.imageType == 'C':
                bigPolyFeatures = np.row_stack((bigPolyFeatures, trainingSample.polyFeatures))
                bigRD = np.append(bigRD, trainingSample.RD)
            elif info.imageType == 'D':
                tempPolyFeatures = np.column_stack((trainingSample.polyFeatures[:,0:25], trainingSample.polyFeatures[:,26]))
                bigPolyFeatures = np.row_stack((bigPolyFeatures, tempPolyFeatures))
                bigRD = np.append(bigRD, trainingSample.RD)

    #Create least-squares regression model
    regr = linear_model.LinearRegression()
    regr.fit(bigPolyFeatures, bigRD)
    
    #Extract resultant theta values
    if info.imageType == 'C':
        theta = regr.coef_
    elif info.imageType == 'D':
        theta = np.zeros((trainingSample.polyFeatures.shape[1]))
        theta[0:24] = regr.coef_[0:24]
        theta[26] = regr.coef_[25]
    
    #Store the generated model
    trainingModels.append(theta.copy())

#Save the end models and the matched cValue order array
np.save(dir_TrainingResults + 'cValues', cValues)
pickle.dump(trainingSamples, open(dir_TrainingResults + 'trainingSamples.p', 'wb'))
np.save(dir_TrainingResults + 'trainedModels', trainingModels)





In [ ]:
print('\n\n\n\n' + ('-' * int(consoleColumns)))
print('DETERMINING BEST C')
print('-' * int(consoleColumns) + '\n')

#Load pre-trained data
cValues = np.load(dir_TrainingResults + 'cValues.npy')
trainingSamples = pickle.load(open(dir_TrainingResults + 'trainingSamples.p', 'rb'))
trainingModels = np.load(dir_TrainingResults + 'trainedModels.npy')

areaUnderCurveList = []
#For each of the proposed c values
for cNum in tqdm(range(0, len(cValues)), desc = 'c Values', leave = True):
    theta = trainingModels[cNum]
    
    #Determine the sum total distortion remaining in the samples
    areaUnderCurve = 0

    #Run SLADS on all of the samples
    results = Parallel(n_jobs=num_threads)(delayed(runSLADS)(info, trainingSamples[sampleNum], trainingSamples[sampleNum].maskObject, theta, stopPerc, simulationFlag=True, trainPlotFlag=False, animationFlag=False) for sampleNum in tqdm(range(0,len(trainingSamples)), desc = 'Training Samples', leave = True))

    #Add resulting area under the curve to the rolling sum
    for result in results: areaUnderCurve += np.trapz(result.TDList, result.percMeasuredList)
    
    #for sampleNum in tqdm(range(0,len(trainingSamples)), desc = 'Training Samples', leave = True):
        
        #Run SLADS till its stopping condition has been met
        #result = runSLADS(info, trainingSamples[sampleNum], trainingSamples[sampleNum].maskObject, theta, stopPerc, simulationFlag=True, trainPlotFlag=False, animationFlag=False)
        
        #Add on the area under the curve to the rolling sum
        #areaUnderCurve = areaUnderCurve + np.trapz(result.TDList, result.percMeasuredList)

    #Append the total area under the curve
    areaUnderCurveList.append(areaUnderCurve)

#Select the c value and corresponding theta set that minimizes the summed total distortion across the samples
bestIndex = np.argmin(areaUnderCurveList)
bestC = cValues[bestIndex]
bestTheta = trainingModels[bestIndex]

# Find the Threshold on stopping condition that corresponds to the desired total distortion (TD) value set above
if findStopThresh:   
    sys.error('ERROR! - Automatic determination of a stopping threshold has not yet been fully implemented!')
    #threshold = findStoppingThreshold(trainingDataPath,NumImagesForSLADS,Best_c,PercentageInitialMask,DesiredTD,reconPercVector,SizeImage)
    #np.save(dir_TrainingResults + 'foundThreshold', threshold) 

#Save the best model
np.save(dir_TrainingResults + 'bestC', bestC)
np.save(dir_TrainingResults + 'bestTheta', bestTheta)

#Load the best model
bestC = np.load(dir_TrainingResults + 'bestC.npy')
bestTheta = np.load(dir_TrainingResults + 'bestTheta.npy')

In [ ]:
print('\n\n' + ('#' * int(consoleColumns)))
print('PLOT TRAINING CONVERGENCE ')
print(('#' * int(consoleColumns)) + '\n')
#Not implemented at this time

print('\n\n' + ('#' * int(consoleColumns)))
print('PERFORMING TESTING')
print(('#' * int(consoleColumns)) + '\n')

testingSamples = []
sortedTestingSampleFolders = natsort.natsorted(glob.glob(dir_TestingData + '/*'), reverse=False) 

#Run SLADS on each of the testing samples
results = Parallel(n_jobs=num_threads)(delayed(testing_parhelper)(sortedTestingSampleFolders[sampleNum], info, bestTheta, stopPerc, animationGeneration) for sampleNum in tqdm(range(0,len(sortedTestingSampleFolders)), desc = 'Testing Images', leave = True))
    
#Create holding arrays for all of the results
MSE_testingResults = []
SSIM_testingResults = []
TD_testingResults = []
perc_testingResults = []

#Extract results from returned object; for every line scanned there is a value
for result in results: 
    MSE_testingResults.append(result.MSEList)
    SSIM_testingResults.append(result.SSIMList)
    TD_testingResults.append(result.TDList)
    perc_testingResults.append(result.percMeasuredList)

precision = 0.01

percents, testingSSIM_mean = percResults(SSIM_testingResults, perc_testingResults, precision)

#Save average SSIM per percentage data
np.savetxt(dir_TestingResults+'testingAverageSSIM_Percentage.csv', np.transpose([percents, testingSSIM_mean]), delimiter=',')

#Save average SSIM per percentage data in plot
font = {'size' : 18}
plt.rc('font', **font)
f = plt.figure(figsize=(20,8))
ax1 = f.add_subplot(1,1,1)    
ax1.plot(percents, testingSSIM_mean,color='black') 
ax1.set_xlabel('% Pixels Measured')
ax1.set_ylabel('Average SSIM')
plt.savefig(dir_TestingResults + 'testingAverageSSIM_Percentage' + '.png')
plt.close()

#Find the final results for each image
lastSSIMResult = []
lastMSEResult = []
lastTDResult = []
percLinesScanned = []
percPixelsScanned = []
for i in range(0, len(SSIM_testingResults)):
    lastSSIMResult.append(SSIM_testingResults[i][len(SSIM_testingResults[i])-1])
    lastMSEResult.append(MSE_testingResults[i][len(MSE_testingResults[i])-1])
    lastTDResult.append(TD_testingResults[i][len(TD_testingResults[i])-1])
    percLinesScanned.append((len(SSIM_testingResults[i])/len(maskObject.linesToScan))*100)
    percPixelsScanned.append(perc_testingResults[i][len(perc_testingResults[i])-1])

#Printout final results 
dataPrintout = []
dataPrintout.append(['Average SSIM:', np.mean(lastSSIMResult), '+/-', np.std(lastSSIMResult)])
dataPrintout.append(['Average MSE:', np.mean(lastMSEResult), '+/-', np.std(lastMSEResult)])
dataPrintout.append(['Average TD:', np.mean(lastTDResult), '+/-', np.std(lastTDResult)])
dataPrintout.append(['Average % Lines Scanned:', np.mean(percLinesScanned),'+/-', np.std(percLinesScanned)])
dataPrintout.append(['Average % Pixels Scanned:', np.mean(percPixelsScanned),'+/-',np.std(percPixelsScanned)])
pd.DataFrame(dataPrintout).to_csv(dir_TestingResults + 'dataPrintout.csv')

#AFTER INTENDED PROCEDURES (TRAINING/TESTING) HAVE BEEN PERFORMED
print('\n\n\n' + ('#' * int(consoleColumns)))
print('PROGRAM COMPLETE')
print('#' * int(consoleColumns) + '\n')



#NonParallized testing Version; keep for troubleshooting purposes
results = []
for sampleNum in tqdm(range(0,len(sortedTestingSampleFolders)), desc = 'Testing Samples', leave = True):
    testingSampleFolder = sortedTestingSampleFolders[sampleNum]
    dataSampleName = os.path.basename(testingSampleFolder)
    
    #Obtain testing images
    images = []
    for imageFileName in natsort.natsorted(glob.glob(testingSampleFolder + '/*.' + 'tiff'), reverse=False):
        #Temporary solution for reading in the .TIFF files; replace with reading direct from a csv...
        images.append(Image.fromarray((libtiff.TIFF.open(imageFileName).read_image()*255.0).astype('uint8')).convert('L') )
    
    #Read in the width and height; when RAW files are supported
    #width, height = pd.read_csv(testingSampleFolder+'/dimensions.csv', sep=',', header=None).values[0].tolist()
    width, height = images[0].size
    
    #Create a new maskObject
    maskObject = MaskObject(width, height, measurementPercs=[])
    
    #How should the mz ranges be weighted (all equal for now)
    mzWeights = np.ones(len(desiredMassRanges))/len(desiredMassRanges)
    
    #Define information as a new Sample object
    testingSample = Sample(dataSampleName, images, desiredMassRanges, maskObject, mzWeights, dir_TestingResults)
    
    #Run SLADS till its stopping condition has been met
    result = runSLADS(info, testingSample, maskObject, bestTheta, stopPerc, simulationFlag=True, trainPlotFlag=False, animationFlag=animationGeneration)
    
    #Store the result
    results.append(result)